In [1]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import os
from dataparser import parse_json_file
from modeltrain import city_similarity_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20022 entries, 0 to 20021
Data columns (total 7 columns):
session_id        20022 non-null object
unix_timestamp    20022 non-null datetime64[ns]
cities            20022 non-null object
country           20022 non-null category
joining_date      20022 non-null datetime64[ns]
user_id           20022 non-null int64
access_date       20022 non-null object
dtypes: category(1), datetime64[ns](2), int64(1), object(3)
memory usage: 958.5+ KB


In [3]:
file_path = 'city_search.json'
df = parse_json_file(file_path)
similarity_matrix = city_similarity_matrix(df)
similarity_matrix.head()


,Greensboro NC,Plano TX,Kitchener ON,Indianapolis IN,Arlington TX,New Orleans LA,Lincoln NE,Houston TX,Boston MA,San Antonio TX,...,Bakersfield CA,Portland OR,Glendale AZ,Phoenix AZ,Dallas TX,Wichita KS,Nashville TN,Cleveland OH,Tucson AZ,Birmingham AL
Greensboro NC,1.000000,0.000000,0.000000,0.012758,0.000000,0.0,0.0,0.000000,0.024898,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140569,0.000000,0.0,0.000000
Plano TX,0.000000,1.000000,0.006430,0.013871,0.214776,0.0,0.0,0.085575,0.021939,0.081852,...,0.008023,0.005578,0.005689,0.037929,0.292503,0.058072,0.000000,0.000000,0.0,0.000000
Kitchener ON,0.000000,0.006430,1.000000,0.014524,0.003426,0.0,0.0,0.050186,0.004324,0.046886,...,0.012107,0.023635,0.016601,0.038693,0.032241,0.000000,0.000000,0.017877,0.0,0.000000
Indianapolis IN,0.012758,0.013871,0.014524,1.000000,0.011877,0.0,0.0,0.079816,0.037545,0.059465,...,0.009697,0.044957,0.028255,0.075237,0.057615,0.010190,0.049964,0.017448,0.0,0.017345
Arlington TX,0.000000,0.214776,0.003426,0.011877,1.000000,0.0,0.0,0.112165,0.021673,0.120430,...,0.007717,0.009524,0.011113,0.036523,0.228542,0.000000,0.000000,0.000000,0.0,0.000000


In [4]:
def city_recommend(searched_cities_str):
    searched_cities_list = []
    for ele in searched_cities_str.split(','):
        searched_cities_list.append(ele.strip())
    print('searched_cities_list: ', searched_cities_list)   

    #Generate a user vector for searched cities
    length = len(similarity_matrix.columns)
    user_arr = np.zeros((length,))
    #print('user_arr: ', user_arr)
    user_vector = pd.Series(user_arr, index = similarity_matrix.columns)
    for item in user_vector.index:
        if item in searched_cities_str:
            user_vector[item] = 1

    # Normalized the user vector
    magnitude = np.sqrt(np.square(user_vector).sum())
    user_vector = user_vector.divide(magnitude)

    # Calculate the score.
    score = similarity_matrix.dot(user_vector).div(similarity_matrix.sum(axis=1))

    # Drop the searched cities
    score = score.drop(searched_cities_list)
    
    return score.nlargest(10).index.values 


# Test a search query
searched_cities_str = 'Dallas TX, New York NY'
recommend_cities_list = city_recommend(searched_cities_str)

print('Recommended cities:\n', recommend_cities_list)

searched_cities_list:  ['Dallas TX', 'New York NY']
Recommended cities:
 ['Plano TX' 'Arlington TX' 'Fort Worth TX' 'Jersey City NJ' 'Houston TX'
 'Toronto ON' 'Montreal QC' 'Newark NJ' 'Los Angeles CA' 'Philadelphia PA']
